# Agregación y agrupación

Una pieza esencial del análisis de datos de gran tamaño es el resumen eficiente (extracción de información de valor): calcular agregaciones como ``sum()``, ``mean()``, ``median()``, ``min()`` y `` max()``, en el que un solo número da una idea de la naturaleza de un conjunto de datos potencialmente grande (distribución, posición, etc).
En esta sección, exploraremos agregaciones en `Pandas`, desde operaciones simples similares a las que hemos visto en matrices `NumPy`, hasta operaciones más sofisticadas basadas en el concepto de `"groupby"`.

Por conveniencia, usaremos la misma función mágica ``display`` que hemos visto en secciones anteriores:

In [62]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args

    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)

    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## DataFrame Planets

Aquí usaremos el conjunto de datos Planets, disponible a través de la librería [seaborn](http://seaborn.pydata.org/).
El DataFrame proporciona información sobre planetas que los astrónomos han descubierto alrededor de otras estrellas (conocidos como *planetas extrasolares* o *exoplanetas* para abreviar). Se puede descargar con un simple comando de `seaborn`:

In [63]:
# !pip install seaborn

In [64]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [65]:
planets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [66]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


Contiene algunos detalles sobre los más de 1.000 planetas extrasolares descubiertos hasta 2014.

## Agregaciones básicas en Pandas

Anteriormente, exploramos algunas de las agregaciones de datos disponibles para matrices NumPy.
Al igual que con una matriz NumPy unidimensional, para una ``Serie`` de ``Pandas`` los agregados devuelven un valor único:

In [67]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [68]:
ser.sum()

2.811925491708157

In [69]:
ser.mean()

0.5623850983416314

Para un ``DataFrame``, de forma predeterminada los agregados devuelven resultados dentro de cada columna:

In [70]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [71]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

Al especificar el argumento ``axis``, puedes agregar dentro de cada fila:

In [72]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

In [73]:
df.mean(axis=1)

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Las ``Series`` y ``DataFrames`` de Pandas incluyen todas las agregaciones básicas mencionadas en el apartado de Numpy. Además, existe un método conveniente ``describe()`` que calcula varios agregados básicos para cada columna y devuelve el resultado.
Usemos esto en los datos de Planets, por ahora eliminando filas con valores faltantes:

In [74]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


Esta puede ser una forma útil de comenzar a comprender las propiedades generales de un conjunto de datos.
Por ejemplo, vemos en la columna ``year`` que, aunque los exoplanetas se descubrieron ya en 1989, la mitad de todos los expolanets conocidos no se descubrieron hasta 2010 o después.
Esto se debe en gran parte a la misión *Kepler*, que es un telescopio espacial diseñado específicamente para encontrar planetas eclipsantes alrededor de otras estrellas.

La siguiente tabla resume algunas otras agregaciones integradas de Pandas:

| Agregación               | Descripción                     |
|--------------------------|---------------------------------|
| ``count()``              | Número total de items           |
| ``first()``, ``last()``  | Primer y último item            |
| ``mean()``, ``median()`` | Media y mediana                 |
| ``min()``, ``max()``     | Mínimo y máximo                 |
| ``std()``, ``var()``     | Desviación estandar y varianza  |
| ``mad()``                | Desviación absoluta media       |
| ``prod()``               | Producto de los items           |
| ``sum()``                | Suma de los items               |

Todos estos son métodos de objetos ``DataFrame`` y ``Series``.

Sin embargo, para profundizar en los datos, las agregaciones básicas a menudo no son suficientes.
El siguiente nivel de resumen de datos es la operación ``groupby``, que le permite calcular agregaciones de forma rápida y eficiente en subconjuntos de datos.

## GroupBy: Split, Apply, Combine

Las agregaciones básicas pueden darle una idea de su conjunto de datos, pero a menudo preferimos agregar condicionalmente en alguna etiqueta o índice: esto se implementa en la llamada operación ``groupby``.
El nombre "group by" proviene de un comando en el lenguaje de base de datos SQL, pero quizás sea más ilustrativo pensar en los términos acuñados por primera vez por Hadley Wickham de Rstats: *dividir, aplicar, combinar*.

### Split, apply, combine

Un ejemplo canónico de operación de split-apply-combine, donde "apply" es una agregación sumatoria.

Esto deja claro lo que logra el ``groupby``:

- El paso *split* implica dividir y agrupar un ``DataFrame`` dependiendo del valor de la clave especificada.
- El paso *apply* implica calcular alguna función, generalmente un agregado, una transformación o un filtrado, dentro de los grupos individuales.
- El paso *combine* fusiona los resultados de estas operaciones en una matriz de salida.

Si bien esto ciertamente se podría hacer manualmente usando alguna combinación de los comandos de enmascaramiento, agregación y fusión cubiertos anteriormente, una comprensión importante es que *no es necesario crear instancias explícitas de las divisiones intermedias*. Más bien, ``GroupBy`` puede (a menudo) hacer esto en una sola pasada sobre los datos, actualizando la suma, media, recuento, mínimo u otro agregado para cada grupo a lo largo del camino.
El poder de ``GroupBy`` es que abstrae estos pasos: el usuario no necesita pensar en *cómo* se realiza el cálculo, sino que piensa en la *operación como un todo*.

Como ejemplo concreto, veamos el uso de Pandas para el cálculo que se muestra en este diagrama.
Comenzaremos creando la entrada ``DataFrame``:

In [75]:
import pandas as pd

In [76]:
df = pd.DataFrame({'department': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'VV': range(6)})
df

,department,VV
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de split-apply-combine se puede calcular con el método ``groupby()`` de ``DataFrames``, pasando el nombre de la columna clave deseada:

In [77]:
df_temp = df.groupby('department').sum()
df_temp

,VV
department,
A,3
B,5
C,7


Observe que lo que se devuelve no es un conjunto de ``DataFrames``, sino un objeto ``DataFrameGroupBy``.
Este objeto es donde está la magia: puede considerarlo como una vista especial del ``DataFrame``, que está preparado para profundizar en los grupos pero no realiza ningún cálculo real hasta que se aplica la agregación.
Este enfoque de "evaluación diferida" significa que los agregados comunes se pueden implementar de manera muy eficiente de una manera casi transparente para el usuario.

Para producir un resultado, podemos aplicar un agregado a este objeto ``DataFrameGroupBy``, que realizará los pasos apropiados de aplicación/combinación para producir el resultado deseado:

In [78]:
df_grouped = df.groupby('department').mean()
df_grouped

,VV
department,
A,1.5
B,2.5
C,3.5


El método ``sum()`` es sólo una posibilidad; puede aplicar prácticamente cualquier función de agregación común de Pandas o NumPy, así como prácticamente cualquier operación ``DataFrame`` válida, como veremos en la siguiente discusión.

### El objeto GroupBy

El objeto ``GroupBy`` es una abstracción muy flexible.
En muchos sentidos, puedes simplemente tratarlo como si fuera una colección de ``DataFrames`` y hace cálculos complejos. Veamos algunos ejemplos usando los datos de Planets.

Quizás las operaciones más importantes disponibles mediante ``GroupBy`` son *aggregate*, *filter*, *transform* y *apply*.
Analizaremos cada uno de estos más detalladamente en "Aggregate, Filter, Transform, Apply", pero antes de eso, presentemos algunas de las otras funciones que se pueden usar con la operación básica ``GroupBy``.

#### Indexación de columnas

El objeto ``GroupBy`` admite la indexación de columnas de la misma manera que el ``DataFrame`` y devuelve un objeto ``GroupBy`` modificado.
Por ejemplo:

In [79]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [80]:
planets['method'].unique()

array(['Radial Velocity', 'Imaging', 'Eclipse Timing Variations',
       'Transit', 'Astrometry', 'Transit Timing Variations',
       'Orbital Brightness Modulation', 'Microlensing', 'Pulsar Timing',
       'Pulsation Timing Variations'], dtype=object)

In [81]:
planets.groupby('method')

In [82]:
planets.groupby('method')['orbital_period'].mean()

method
Astrometry                          631.180000
Eclipse Timing Variations          4751.644444
Imaging                          118247.737500
Microlensing                       3153.571429
Orbital Brightness Modulation         0.709307
Pulsar Timing                      7343.021201
Pulsation Timing Variations        1170.000000
Radial Velocity                     823.354680
Transit                              21.102073
Transit Timing Variations            79.783500
Name: orbital_period, dtype: float64

In [83]:
planets.groupby('method', as_index=False)[['orbital_period','mass','distance']].mean()

,method,orbital_period,mass,distance
0,Astrometry,631.180000,NaN,17.875000
1,Eclipse Timing Variations,4751.644444,5.125000,315.360000
2,Imaging,118247.737500,NaN,67.715937
3,Microlensing,3153.571429,NaN,4144.000000
4,Orbital Brightness Modulation,0.709307,NaN,1180.000000
5,Pulsar Timing,7343.021201,NaN,1200.000000
6,Pulsation Timing Variations,1170.000000,NaN,NaN
7,Radial Velocity,823.354680,2.630699,51.600208
8,Transit,21.102073,1.470000,599.298080
9,Transit Timing Variations,79.783500,NaN,1104.333333


In [84]:
planets.groupby('method', as_index=False)['orbital_period'].mean()

,method,orbital_period
0,Astrometry,631.180000
1,Eclipse Timing Variations,4751.644444
2,Imaging,118247.737500
3,Microlensing,3153.571429
4,Orbital Brightness Modulation,0.709307
5,Pulsar Timing,7343.021201
6,Pulsation Timing Variations,1170.000000
7,Radial Velocity,823.354680
8,Transit,21.102073
9,Transit Timing Variations,79.783500


Aquí hemos seleccionado un grupo ``Series`` particular del grupo ``DataFrame`` original por referencia a su nombre de columna.
Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamemos a alguna agregación en el objeto:

In [85]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [86]:
len(planets['method'].unique())

10

In [87]:
planets.groupby('method')[['orbital_period']].mean()

,orbital_period
method,
Astrometry,631.180000
Eclipse Timing Variations,4751.644444
Imaging,118247.737500
Microlensing,3153.571429
Orbital Brightness Modulation,0.709307
Pulsar Timing,7343.021201
Pulsation Timing Variations,1170.000000
Radial Velocity,823.354680
Transit,21.102073


In [88]:
planets[planets['method']=='Astrometry'][['orbital_period']].mean()

orbital_period    631.18
dtype: float64

Esto da una idea de la escala general de períodos orbitales (en días) a los que es sensible cada método.

#### Iteración sobre grupos

El objeto ``GroupBy`` admite la iteración directa sobre los grupos, devolviendo cada grupo como una ``Serie`` o un ``DataFrame``:

Esto puede ser útil para hacer ciertas cosas manualmente, aunque a menudo es mucho más rápido usar la funcionalidad incorporada ``apply``, que discutiremos en un momento.

#### Métodos de envío

A través de algo de magia en Python, cualquier método no implementado explícitamente por el objeto ``GroupBy`` será pasado y llamado en los grupos, ya sean objetos ``DataFrame`` o ``Series``.
Por ejemplo, puede utilizar el método ``describe()`` de ``DataFrames`` para realizar un conjunto de agregaciones que describen cada grupo en los datos:

In [89]:
pd.DataFrame(planets.groupby('method')['year'].describe().unstack())

0
      method                               
count Astrometry                        2.0
      Eclipse Timing Variations         9.0
      Imaging                          38.0
      Microlensing                     23.0
      Orbital Brightness Modulation     3.0
...                                     ...
max   Pulsar Timing                  2011.0
      Pulsation Timing Variations    2007.0
      Radial Velocity                2014.0
      Transit                        2014.0
      Transit Timing Variations      2014.0

[80 rows x 1 columns]

Mirar esta tabla nos ayuda a comprender mejor los datos: por ejemplo, la gran mayoría de los planetas han sido descubiertos mediante los métodos de Velocidad Radial y Tránsito, aunque este último solo se volvió común (debido a los telescopios nuevos y más precisos) en la última década. .
Los métodos más nuevos parecen ser la variación del tiempo de tránsito y la modulación del brillo orbital, que no se utilizaron para descubrir un nuevo planeta hasta 2011.

Este es sólo un ejemplo de la utilidad de los métodos de envío.
Observe que se aplican *a cada grupo individual*, y los resultados luego se combinan dentro de ``GroupBy`` y se devuelven.
Nuevamente, cualquier método ``DataFrame``/``Series`` válido se puede usar en el objeto ``GroupBy`` correspondiente, lo que permite algunas operaciones muy flexibles y poderosas.

### Aggregate, filter, transform, apply

La discusión anterior se centró en la agregación para la operación combinada, pero hay más opciones disponibles.
En particular, los objetos ``GroupBy`` tienen métodos ``aggregate()``, ``filter()``, ``transform()`` y ``apply()`` que implementan eficientemente una variedad de funciones útiles. operaciones antes de combinar los datos agrupados.

Para los fines de las siguientes subsecciones, usaremos este ``DataFrame``:

In [90]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'department': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'anio': [2020,2020,2020,2021,2021,2021],
                   'VV': rng.randint(0, 10, 6)},
                   columns = ['department', 'anio', 'VV'])
df

,department,anio,VV
0,A,2020,5
1,B,2020,0
2,C,2020,3
3,A,2021,3
4,B,2021,7
5,C,2021,9


#### Aggregation

Ahora estamos familiarizados con las agregaciones ``GroupBy`` con ``sum()``, ``median()`` y similares, pero el método ``aggregate()`` permite aún más flexibilidad.
Puede tomar una cadena, una función o una lista de las mismas y calcular todos los agregados a la vez.
Aquí hay un ejemplo rápido que combina todo esto:

In [91]:
df.groupby('department').median()

,anio,VV
department,,
A,2020.5,4.0
B,2020.5,3.5
C,2020.5,6.0


In [93]:
df_grouped = df.groupby('department').aggregate(['min', np.median, max])
df_grouped

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18744\1885321942.py:1: FutureWarning: The provided callable <function median at 0x000001EA4977B560> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_grouped = df.groupby('department').aggregate(['min', np.median, max])
C:\Users\Usuario\AppData\Local\Temp\ipykernel_18744\1885321942.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_grouped = df.groupby('department').aggregate(['min', np.median, max])


anio                VV           
             min  median   max min median max
department                                   
A           2020  2020.5  2021   3    4.0   5
B           2020  2020.5  2021   0    3.5   7
C           2020  2020.5  2021   3    6.0   9

Otro patrón útil es pasar un diccionario que asigna nombres de columnas a las operaciones que se aplicarán en esa columna:

In [94]:
planets.groupby('method', as_index=False)[['orbital_period','mass','distance']].aggregate(np.nanmean)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18744\709129802.py:1: FutureWarning: The provided callable <function nanmean at 0x000001EA49782980> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  planets.groupby('method', as_index=False)[['orbital_period','mass','distance']].aggregate(np.nanmean)


,method,orbital_period,mass,distance
0,Astrometry,631.180000,NaN,17.875000
1,Eclipse Timing Variations,4751.644444,5.125000,315.360000
2,Imaging,118247.737500,NaN,67.715937
3,Microlensing,3153.571429,NaN,4144.000000
4,Orbital Brightness Modulation,0.709307,NaN,1180.000000
5,Pulsar Timing,7343.021201,NaN,1200.000000
6,Pulsation Timing Variations,1170.000000,NaN,NaN
7,Radial Velocity,823.354680,2.630699,51.600208
8,Transit,21.102073,1.470000,599.298080
9,Transit Timing Variations,79.783500,NaN,1104.333333


In [96]:
df_grouped = df.groupby('department').aggregate({'anio': 'min',
                                                'VV': 'mean'}).rename(columns={'anio':'anio_min',
                                                                                "VV":"VV_mean"})
df_grouped

,anio_min,VV_mean
department,,
A,2020,4.0
B,2020,3.5
C,2020,6.0


#### Filtering

Una operación de filtrado le permite eliminar datos según las propiedades del grupo.
Por ejemplo, es posible que deseemos mantener todos los grupos en los que la desviación estándar sea mayor que algún valor crítico:

In [97]:
def filter_func(x):
    return x['VV'].min() > 1

In [99]:
filter_func(df)

False

In [100]:
df['VV'] > 0

0     True
1    False
2     True
3     True
4     True
5     True
Name: VV, dtype: bool

In [101]:
df[df['VV'] > 0]

,department,anio,VV
0,A,2020,5
2,C,2020,3
3,A,2021,3
4,B,2021,7
5,C,2021,9


In [102]:
display('df', "df.groupby('department').min()", "df.groupby('department').filter(filter_func)")
# el ultimo filtro no me devulve la categoria que no
# cumple el filtro dado por la funcion filter_func
# .min() > 1. Por ende no sale department B.

df
  department  anio  VV
0          A  2020   5
1          B  2020   0
2          C  2020   3
3          A  2021   3
4          B  2021   7
5          C  2021   9

df.groupby('department').min()
            anio  VV
department          
A           2020   3
B           2020   0
C           2020   3

df.groupby('department').filter(filter_func)
  department  anio  VV
0          A  2020   5
2          C  2020   3
3          A  2021   3
5          C  2021   9

#### Transformation

Si bien la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinarlos.
Para tal transformación, la salida tiene la misma forma que la entrada.
Un ejemplo común es centrar los datos restando la media grupal:

In [103]:
def mi_funcion(x):
    return x - x.mean()

In [105]:
df.groupby('department')['VV'].mean()

department
A    4.0
B    3.5
C    6.0
Name: VV, dtype: float64

In [106]:
# df.groupby('department').transform(lambda x: x - x.mean())
df.groupby('department').transform(mi_funcion)

,anio,VV
0,-0.5,1.0
1,-0.5,-3.5
2,-0.5,-3.0
3,0.5,-1.0
4,0.5,3.5
5,0.5,3.0


In [108]:
df['VV'].mean()

4.5

In [109]:
df['VV_example'] = df['VV'] - df['VV'].mean()
# lo que no hace el transform
df

,department,anio,VV,VV_example
0,A,2020,5,0.5
1,B,2020,0,-4.5
2,C,2020,3,-1.5
3,A,2021,3,-1.5
4,B,2021,7,2.5
5,C,2021,9,4.5


#### The apply() method

El método ``apply()`` le permite aplicar una función arbitraria a los resultados del grupo.
La función debe tomar un ``DataFrame`` y devolver un objeto ``Pandas`` (por ejemplo, ``DataFrame``, ``Series``) o un escalar; la operación combinada se adaptará al tipo de salida devuelta.

Por ejemplo, aquí hay una ``apply()`` que normaliza la primera columna por la suma de la segunda:

In [110]:
def norm_by_data2(x):
    # x is a DataFrame of group values
    # x['data1'] = x['data1'] / x['data2'].sum()
    x['anio'] /= x['VV'].sum()
    return x

display('df', "df.groupby('department').apply(norm_by_data2)")

df
  department  anio  VV  VV_example
0          A  2020   5         0.5
1          B  2020   0        -4.5
2          C  2020   3        -1.5
3          A  2021   3        -1.5
4          B  2021   7         2.5
5          C  2021   9         4.5

df.groupby('department').apply(norm_by_data2)
             department        anio  VV  VV_example
department                                         
A          0          A  252.500000   5         0.5
           3          A  252.625000   3        -1.5
B          1          B  288.571429   0        -4.5
           4          B  288.714286   7         2.5
C          2          C  168.333333   3        -1.5
           5          C  168.416667   9         4.5

In [111]:
df.groupby('department').sum()

,anio,VV,VV_example
department,,,
A,4041,8,-1.0
B,4041,7,-2.0
C,4041,12,3.0


In [112]:
print(3/8)

0.375


``apply()`` dentro de ``GroupBy`` es bastante flexible: el único criterio es que la función tome un ``DataFrame`` y devuelva un objeto o escalar ``Pandas``; ¡Lo que hagas en el medio depende de ti!

### Especificación de la clave del split

En los ejemplos simples presentados anteriormente, dividimos el ``DataFrame`` en un solo nombre de columna.
Esta es sólo una de las muchas opciones mediante las cuales se pueden definir los grupos, y aquí veremos algunas otras opciones para la especificación de grupos.

#### Una lista, array, serie o índice que proporciona las claves de agrupación.

La clave puede ser cualquier serie o lista con una longitud que coincida con la del ``DataFrame``. Por ejemplo:

In [113]:
L = [0, 1, 0, 1, 2, 0]
# cada numero representa un grupo, ejemplo: 0 es un grupo
# dado por las filas 0,2,5. 1 es un grupo que es dado por
# los registros(filas) 1,3. 2 seria dado por la fila 4.
display('df', 'df.groupby(L).sum()')

,department,anio,VV,VV_example
0,A,2020,5,0.5
1,B,2020,0,-4.5
2,C,2020,3,-1.5
3,A,2021,3,-1.5
4,B,2021,7,2.5
5,C,2021,9,4.5
,department,anio,VV,VV_example
0,ACC,6061,17,3.5
1,BA,4041,3,-6.0
2,B,2021,7,2.5


Por supuesto, esto significa que hay otra forma más detallada de lograr el ``df.groupby('key')`` de antes:

In [114]:
display('df', "df.groupby(df['department']).sum()")

df
  department  anio  VV  VV_example
0          A  2020   5         0.5
1          B  2020   0        -4.5
2          C  2020   3        -1.5
3          A  2021   3        -1.5
4          B  2021   7         2.5
5          C  2021   9         4.5

df.groupby(df['department']).sum()
            anio  VV  VV_example
department                      
A           4041   8        -1.0
B           4041   7        -2.0
C           4041  12         3.0

#### Un índice de mapeo de diccionario o serie para un grupo

Otro método consiste en proporcionar un diccionario que asigne valores de índice a las claves del grupo:

In [115]:
df2 = df.set_index('department')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,anio,VV,VV_example
department,,,
A,2020,5,0.5
B,2020,0,-4.5
C,2020,3,-1.5
A,2021,3,-1.5
B,2021,7,2.5
C,2021,9,4.5
,anio,VV,VV_example
department,,,
consonant,8082,19,1.0


#### Cualquier función de Python

De manera similar al mapeo, puede pasar cualquier función de Python que ingrese el valor del índice y genere el grupo de salida:

In [116]:
display('df2', 'df2.groupby(str.lower).mean()')
# cuando no indico una variable en el groupby
# me agrupa por el indice

,anio,VV,VV_example
department,,,
A,2020,5,0.5
B,2020,0,-4.5
C,2020,3,-1.5
A,2021,3,-1.5
B,2021,7,2.5
C,2021,9,4.5
,anio,VV,VV_example
department,,,
a,2020.5,4.0,-0.5


#### Una lista de claves válidas

Además, cualquiera de las opciones clave anteriores se puede combinar para agruparlas en un índice múltiple:

In [117]:
df2.groupby([str.lower, mapping]).mean()

,,anio,VV,VV_example
department,department,,,
a,vowel,2020.5,4.0,-0.5
b,consonant,2020.5,3.5,-1.0
c,consonant,2020.5,6.0,1.5


### Ejemplo de agrupación

Como ejemplo de esto, en un par de líneas de código Python podemos juntar todo esto y contar los planetas descubiertos por método y por década:

In [118]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)
# unstack me devulve los indices uno en fila y otro en
# columna

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


In [125]:
print(decade)
print(planets)

0       2000s
1       2000s
2       2010s
3       2000s
4       2000s
        ...  
1030    2000s
1031    2000s
1032    2000s
1033    2000s
1034    2000s
Name: decade, Length: 1035, dtype: object
               method  number  orbital_period   mass  distance  year
0     Radial Velocity       1      269.300000   7.10     77.40  2006
1     Radial Velocity       1      874.774000   2.21     56.95  2008
2     Radial Velocity       1      763.000000   2.60     19.84  2011
3     Radial Velocity       1      326.030000  19.40    110.62  2007
4     Radial Velocity       1      516.220000  10.50    119.47  2009
...               ...     ...             ...    ...       ...   ...
1030          Transit       1        3.941507    NaN    172.00  2006
1031          Transit       1        2.615864    NaN    148.00  2007
1032          Transit       1        3.191524    NaN    174.00  2007
1033          Transit       1        4.125083    NaN    293.00  2008
1034          Transit       1        4.187757

Esto muestra el poder de combinar muchas de las operaciones que hemos discutido hasta este punto cuando analizamos conjuntos de datos realistas.
¡Inmediatamente obtenemos una comprensión aproximada de cuándo y cómo se han descubierto planetas en las últimas décadas!

Aquí sugeriría profundizar en estas pocas líneas de código y evaluar los pasos individuales para asegurarse de comprender exactamente lo que están haciendo con el resultado.
Sin duda, es un ejemplo algo complicado, pero comprender estos elementos le brindará los medios para explorar sus propios datos de manera similar.